# Multi-Agent Conversation

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## Define an AutoGen agent

In [2]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [3]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure, here's a joke for you:

Why couldn't the bicycle stand up by itself?
Because it was two tired!


In [4]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Certainly! Please provide the joke you would like me to repeat.


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [5]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [6]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hi Joe, nice to meet you! Let's jump into it, shall we? So, I recently joined a gym because I heard it's a great place to meet guys. But so far, all I've met is my personal trainer who seems to think I need to work on my "sense of humor." I guess being told to "drop and give me 20 jokes" wasn't what he meant...

--------------------------------------------------------------------------------
joe (to cathy):

Well, I guess I know what my next workout routine will look like!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, well, who needs squats and push-ups when you can get those abs from laughing, right? Nowadays, I'm starting to think that my love life is a lot like a comedy show. It's a mix of awkward silence, mistaken punchlines, and the occasional heckler yelling out 

## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [7]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': "Hi Joe, nice to meet you! Let's jump into it, shall we? So, I "
             "recently joined a gym because I heard it's a great place to meet "
             "guys. But so far, all I've met is my personal trainer who seems "
             'to think I need to work on my "sense of humor." I guess being '
             'told to "drop and give me 20 jokes" wasn\'t what he meant...',
  'role': 'user'},
 {'content': 'Well, I guess I know what my next workout routine will look '
             'like!',
  'role': 'assistant'},
 {'content': 'Haha, well, who needs squats and push-ups when you can get those '
             "abs from laughing, right? Nowadays, I'm starting to think that "
             "my love life is a lot like a comedy show. It's a mix of awkward "
             'silence, mistaken punchlines, and the occasional heckler yelling '
             'out "I\'ve heard this one before!"',
  'role

In [8]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 167,
                                                             'cost': 0.0004075,
                                                             'prompt_tokens': 314,
                                                             'total_tokens': 481},
                                      'total_cost': 0.0004075},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 167,
                                                             'cost': 0.0004075,
                                                             'prompt_tokens': 314,
                                                             'total_tokens': 481},
                                      'total_cost': 0.0004075}}


In [9]:
pprint.pprint(chat_result.summary)

('Haha, well, who needs squats and push-ups when you can get those abs from '
 "laughing, right? Nowadays, I'm starting to think that my love life is a lot "
 "like a comedy show. It's a mix of awkward silence, mistaken punchlines, and "
 'the occasional heckler yelling out "I\'ve heard this one before!"')


## Get a better summary of the conversation

In [10]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hi Joe, nice to meet you! Let's jump into it, shall we? So, I recently joined a gym because I heard it's a great place to meet guys. But so far, all I've met is my personal trainer who seems to think I need to work on my "sense of humor." I guess being told to "drop and give me 20 jokes" wasn't what he meant...

--------------------------------------------------------------------------------
joe (to cathy):

Well, I guess I know what my next workout routine will look like!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, well, who needs squats and push-ups when you can get those abs from laughing, right? Nowadays, I'm starting to think that my love life is a lot like a comedy show. It's a mix of awkward silence, mistaken punchlines, and the occasional heckler yelling out 

In [11]:
pprint.pprint(chat_result.summary)

('The conversation revolves around jokes and humor in a gym setting, with a '
 'humorous twist related to dating and relationships.')


## Chat Termination

Chat can be terminated using a termination conditions.

In [12]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [13]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, ready to laugh? Why don't scientists trust atoms? Because they make up everything!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, classic one! Here's one for you: Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, I love that one, Joe! Here's another one for you: Why couldn't the bicycle stand up by itself? It was two tired!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! You're on a roll. How about this: How do you organize a space party? You planet!

--------------------------------------------------------------------------------
cathy (to joe):

Ha

In [14]:
cathy.send(message="Wh at's last joke we talked about?", recipient=joe)

cathy (to joe):

Wh at's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was: What do you call fake spaghetti? An impasta! It was a good one!

--------------------------------------------------------------------------------
cathy (to joe):

Thanks for playing along! That was a great joke. I gotta go now. Take care, Joe!

--------------------------------------------------------------------------------
